In [ ]:
'''
Image recognition was initially trained using the wrong class IDs
Hence, we need to adjust what we output and remap the right class IDs because of my blunder.
'''

In [ ]:
import json

with open('mdpfiles\\predicted_output\\result.json') as json_file:
    data = json.load(json_file)

In [ ]:
all_images = []

for each in data:
    name = each['objects'][0]['name']
    x_axis = each['objects'][0]['relative_coordinates']['left_x']
    y_axis = each['objects'][0]['relative_coordinates']['top_y']
    confidence = each['objects'][0]['confidence']
    image_info = [name, x_axis, y_axis, confidence]
    all_images.append(image_info)

In [ ]:
import pandas as pd
import numpy as np

columns = [
    'Description',
    'BBox X-axis',
    'BBox Y-axis',
    'Confidence'
]

df = pd.DataFrame(data = all_images, columns = columns) 

In [ ]:
'''
Import helper function to get each respective image's path
'''

from filelister import *

dir_name = 'mdpfiles\\samples'
image_list = getFileList(dir_name)

In [ ]:
'''
Declare columns for image_df
'''
columns_image = [
    'Image Width',
    'Image Height'
]

image_df = pd.DataFrame(columns = columns_image)

In [ ]:
from PIL import Image

image_meta = []

'''
For loop to iterate through image paths in image_list
Then, run Image function to get width and height for each image
'''
for image_path in image_list: # I need to load each image name into a list
    im = Image.open(image_path)
    img_width = im.size[0]
    img_height = im.size[1]
    image_df.loc[len(image_df)]=[img_width, img_height]

In [ ]:
'''
Combine original df with image_df to store all meta info together
'''
final_df = pd.concat([df, image_df], axis=1, sort=False)

In [ ]:
'''
Will need to map a dictionary of values to distances (in terms of how many blocks away)
For now, assume that the blocks away = 0
In future, assume we need to map different bounds for:
0
1
2
3
4
5
6
7
8
9
''' 

final_df['Left Bound'] = (final_df['Image Width']/3) * 1 
final_df['Center Bound'] = (final_df['Image Width']/3) * 2
final_df['Right Bound'] = final_df['Image Width']
final_df['Offset Factor'] = 0

In [ ]:
'''
To determine most realistic partition for bounding box

For Left Bounding box
Img X-axis + bounding box length

For Right Bounding box
Img X-axis 

But for now do simplest implementation without bounding box factor

'''

In [ ]:
offset_list = []

for index, row in final_df.iterrows():
    if (row['BBox X-axis'] < row['Left Bound']):
        offset_list.append('Left') 
    elif (row['BBox X-axis'] > row['Center Bound']): 
        offset_list.append('Right')
    else:
        offset_list.append('Center')
        
final_df['Offset Factor'] = offset_list

In [ ]:
final_df

In [ ]:
final_df = final_df[[
    'Description',
    'Offset Factor',
    'Confidence',
]]
final_df

In [ ]:
'''
need dictionary mapping for description to actual raw values for mdf format
'''